## ***Competition Submission of Titanic***

In [62]:
import pandas as pd
import numpy as np
# classifiers
from xgboost import XGBClassifier
import xgboost as xgb
# egitim verisini ayirma
from sklearn.model_selection import train_test_split
# classifiers, regressors
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC, SVR
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
import xgboost as xgb
# pipeline meta model
from sklearn.pipeline import Pipeline
# imbalancy elimination
from imblearn.over_sampling import ADASYN

# uyarilari dikkate alma
import warnings
warnings.filterwarnings('ignore')

# pandas varsayilan olarak cok sayida sutun veya satir varsa tumunu gostermez
# bu nedenle 100 sutun ve satir gostermesi icin
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

# virgulden sonra belirli sayida basamak gosterimi icin 
pd.options.display.float_format = '{:,.3f}'.format
# aciklama sutunun tam cikmasi icin
pd.options.display.max_colwidth = 100

# font tanimlamalari
font_title = {'family': 'times new roman', 'color': 'darkred', 
              'weight': 'bold', 'size': 14}
font_axis  = {'family': 'times new roman', 'color': 'darkred', 
              'weight': 'bold', 'size': 14}

In [63]:
test = pd.read_csv('data/test.csv')

test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [64]:
test = test.drop(['Cabin','Ticket'], axis=1)

In [65]:
def add_title(col):
    if 'Mrs' in col:
        return 'mrs'
    elif 'Master' in col:
        return 'master'
    elif 'Mr' in col:
        return 'mr'
    elif 'Miss' in col:
        return 'miss'
    else:
        return 'no_title'

In [66]:
add_title('Fortune, Mr. Charles Alexander')

'mr'

In [67]:
test['title'] = test['Name'].apply(add_title)

In [68]:
test['title'].unique()

array(['mr', 'mrs', 'miss', 'master', 'no_title'], dtype=object)

In [69]:
# cinsiyet, liman ve title bilgisi get_dummies metodu ile encode edilir
test = pd.concat([test, pd.get_dummies(test.Sex, 
                                             drop_first=True, prefix='gender')], axis=1)
test = pd.concat([test, pd.get_dummies(test.Embarked, 
                                             drop_first=True, prefix='embarked')], axis=1)

test = pd.concat([test,pd.get_dummies(test['title'])], axis=1)

In [70]:
# encode edilen ve gereksiz sutunlar drop edilir
test.drop(['Name', 'Sex', 'Embarked',  'title', 'no_title'], axis=1, inplace=True)

In [71]:
# unique deger sayilari
type_of_vars = {}
categoric = []
continuous = []
cols_df = pd.DataFrame([])
for col in test.columns:
    type_of_vars['col_name'] = col       
    type_of_vars['type'] = 'continuous' if test[col].nunique()>20 else 'categoric'
    type_of_vars['n_unique'] = test[col].nunique()
    type_of_vars['unique'] = test[col].unique()
    cols_df = pd.concat([cols_df,pd.DataFrame([type_of_vars])],)
    cols_df = cols_df.sort_values(by='n_unique')
    continuous.append(col) if test[col].nunique()>20 else categoric.append(col)
pd.set_option('display.max_colwidth', 200)
display(cols_df)

,col_name,type,n_unique,unique
0,gender_male,categoric,2,"[1, 0]"
0,embarked_Q,categoric,2,"[1, 0]"
0,embarked_S,categoric,2,"[0, 1]"
0,master,categoric,2,"[0, 1]"
0,miss,categoric,2,"[0, 1]"
0,mr,categoric,2,"[1, 0]"
0,mrs,categoric,2,"[0, 1]"
0,Pclass,categoric,3,"[3, 2, 1]"
0,SibSp,categoric,7,"[0, 1, 2, 3, 4, 5, 8]"
0,Parch,categoric,8,"[0, 1, 3, 2, 4, 6, 5, 9]"


In [72]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Age          332 non-null    float64
 3   SibSp        418 non-null    int64  
 4   Parch        418 non-null    int64  
 5   Fare         417 non-null    float64
 6   gender_male  418 non-null    uint8  
 7   embarked_Q   418 non-null    uint8  
 8   embarked_S   418 non-null    uint8  
 9   master       418 non-null    uint8  
 10  miss         418 non-null    uint8  
 11  mr           418 non-null    uint8  
 12  mrs          418 non-null    uint8  
dtypes: float64(2), int64(4), uint8(7)
memory usage: 22.6 KB


In [73]:
test[test.Fare.isnull()]

,PassengerId,Pclass,Age,SibSp,Parch,Fare,gender_male,embarked_Q,embarked_S,master,miss,mr,mrs
152,1044,3,60.500,0,0,nan,1,0,1,0,0,1,0


In [74]:
test[['Fare']].describe()

,Fare
count,417.000
mean,35.627
std,55.908
min,0.000
25%,7.896
50%,14.454
75%,31.500
max,512.329


In [75]:
test.Fare.fillna(test.Fare.median(), inplace=True)

---
### ***Yas (Age) Sutunundaki Eksik Degerleri Tahmin Ile Doldurma***

In [76]:
# yas degerleri dolu ve bos olan sample lar ayri ayri alinir
test_w_age = test[test['Age'].notnull()]
test_no_age = test[test['Age'].isnull()]

In [77]:
test_w_age

,PassengerId,Pclass,Age,SibSp,Parch,Fare,gender_male,embarked_Q,embarked_S,master,miss,mr,mrs
0,892,3,34.500,0,0,7.829,1,1,0,0,0,1,0
1,893,3,47.000,1,0,7.000,0,0,1,0,0,0,1
2,894,2,62.000,0,0,9.688,1,1,0,0,0,1,0
3,895,3,27.000,0,0,8.662,1,0,1,0,0,1,0
4,896,3,22.000,1,1,12.287,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,1301,3,3.000,1,1,13.775,0,0,1,0,1,0,0
411,1303,1,37.000,1,0,90.000,0,1,0,0,0,0,1
412,1304,3,28.000,0,0,7.775,0,0,1,0,1,0,0
414,1306,1,39.000,0,0,108.900,0,0,0,0,0,0,0


#### ***Model - Pipeline***
Meta model olarak dusunulebilecek pipeline sinifi ile modelleri test ederek en iyi performans veren modeli sececegiz.

In [78]:
# hedef degisken
y = test_w_age['Age']
# aciklayici degiskenler
X = test_w_age.drop(['PassengerId', 'Age'], axis=1)

In [79]:
# egitim ve test kumesini ayirma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [21]:
l = list(np.logspace(-3,2,6))
l.append('scale')
l

[0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 'scale']

In [ ]:
%%time
# hucrenin calisma suresini tutacak

# pipeline tanimla
# ilk model knn ve preprocessing yapilmayacak 
pipe = Pipeline([('regressor', KNeighborsRegressor())])

# aranacak model ve parametreleri belirle
param_grid = [{'regressor': [KNeighborsRegressor()],
               'regressor__n_neighbors': np.arange(3,10),
               'regressor__weights': ['uniform', 'distance']},
              {'regressor': [LinearRegression()]},
              {'regressor': [Ridge()], 
               'regressor__alpha': np.logspace(-3,3,7)},
              {'regressor': [Lasso()], 
               'regressor__alpha': np.logspace(-3,3,7)},
              {'regressor': [DecisionTreeRegressor()], 
               'regressor__max_depth': np.arange(3,10)},              
              {'regressor': [RandomForestRegressor()], 
               'regressor__max_depth': np.arange(3,10)},
              {'regressor': [SVR()], 
               'regressor__gamma': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 'scale'],
               'regressor__C': np.logspace(-3,2,6)},
              {'regressor': [SVR(kernel='linear')], 
               'regressor__C': np.logspace(-3,2,6)},
              {'regressor': [XGBRegressor()],
               'regressor__gamma': [0, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0],
               'regressor__reg_alpha': [0, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0], 
               'regressor__reg_lambda': np.logspace(-3,2,6), 
               'regressor__max_depth': np.arange(3,10)},
              {'regressor': [LGBMRegressor()],
               'regressor__reg_alpha': [0, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0], 
               'regressor__reg_lambda': [0, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0], 
               'regressor__max_depth': np.arange(3,10)}              
             ]
# gridsearch nesnesi tanimla
grid = GridSearchCV(estimator=pipe, param_grid=param_grid, scoring=['neg_root_mean_squared_error','r2'], 
                    cv=7, n_jobs=2, return_train_score=True, refit='neg_root_mean_squared_error')
# modeli egit
grid.fit(X_train, y_train)

# sonuclar
print("Best params: \n{}\n".format(grid.best_params_))
print("Best cross-validation score: \n{:.2f}\n".format(grid.best_score_))
print("Test-set score: \n{:.2f}\n".format(grid.score(X_test, y_test)))


In [ ]:
# sonuclari df olarak kaydet
grid_df = pd.DataFrame(grid.cv_results_)
grid_df.head(1)

In [ ]:
# sirala
grid_df = grid_df.sort_values(by='mean_test_neg_root_mean_squared_error', ascending=False)
# ilk siradakileri goster
grid_df.loc[:,['param_regressor','params','mean_train_neg_root_mean_squared_error',
               'mean_test_neg_root_mean_squared_error','mean_train_r2','mean_test_r2']].head(5)

In [ ]:
grid_df.loc[1882, 'params']

In [82]:
# secilen modeli olustur ve test et
xgb_reg = XGBRegressor(max_depth=3, reg_alpha=1, reg_lambda=1, gamma=100)
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_test)
# degerleri yuvarla
y_pred = np.around(y_pred)
print('rmse : {:.2f}'.format(np.sqrt(mean_squared_error(y_test, y_pred))))

rmse : 10.41


In [83]:
# secilen modeli butun veriyle egitip tahmin degerleri ile eksik degerleri doldur
xgb_reg = XGBRegressor(max_depth=3, reg_alpha=1, reg_lambda=1, gamma=100)
xgb_reg.fit(X, y)
# eksik degerlerin aciklayici degiskenler bolumu
X_fill = test_no_age.drop(['PassengerId', 'Age'], axis=1)
# eksik degerleri  tahmin ile doldur
predicted_age = xgb_reg.predict(X_fill)
# degerleri incele
predicted_age[:10]

array([26.744122, 42.98792 , 28.840216, 29.816168, 21.118801, 27.755154,
       41.212204, 29.314108, 31.940126, 28.840216], dtype=float32)

In [84]:
# degerleri yuvarlayarak doldur
test_no_age['Age'] = np.around(predicted_age)

In [85]:
# ayirdigimiz dfleri birlestir
test = pd.concat([test_w_age, test_no_age])

In [86]:
# sirala
test = test.sort_values(by='PassengerId')

In [97]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 0 to 417
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Age          418 non-null    float64
 3   SibSp        418 non-null    int64  
 4   Parch        418 non-null    int64  
 5   Fare         418 non-null    float64
 6   gender_male  418 non-null    uint8  
 7   embarked_Q   418 non-null    uint8  
 8   embarked_S   418 non-null    uint8  
 9   master       418 non-null    uint8  
 10  miss         418 non-null    uint8  
 11  mr           418 non-null    uint8  
 12  mrs          418 non-null    uint8  
dtypes: float64(2), int64(4), uint8(7)
memory usage: 25.7 KB


In [88]:
# save the results
test.to_csv('data/titanic_test_clean.csv', index=False)

In [89]:
# yas degiskeninin surekli oldugu veri seti secildi, orada model daha iyi performas gosteriyor
train = pd.read_csv('data/titanic_clean_with_age.csv')

In [96]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Age          891 non-null    float64
 4   SibSp        891 non-null    int64  
 5   Parch        891 non-null    int64  
 6   Fare         891 non-null    float64
 7   gender_male  891 non-null    int64  
 8   embarked_Q   891 non-null    int64  
 9   embarked_S   891 non-null    int64  
 10  master       891 non-null    int64  
 11  miss         891 non-null    int64  
 12  mr           891 non-null    int64  
 13  mrs          891 non-null    int64  
dtypes: float64(2), int64(12)
memory usage: 97.6 KB


In [60]:
def make_submission(model):
    X = train.drop(['PassengerId', 'Survived'], axis=1)
    y = train['Survived']
    X_adasyn, y_adasyn = ADASYN().fit_sample(X,y)
    model.fit(X_adasyn, y_adasyn)
    X_test = test.drop('PassengerId', axis=1)
    y_pred = model.predict(X_test)
    test_predicted = test.copy()
    test_predicted['Survived'] = y_pred
    test_predicted[['PassengerId','Survived']].to_csv('data/submission.csv', index=False)

In [94]:
# secilen model
xgb_model = XGBClassifier(
    learning_rate=0.1,
    n_estimators=44,
    max_depth=8,
    min_child_weight=2,
    gamma=0.2,
    subsample=0.55,
    colsample_bytree=0.55,
    reg_alpha=0,
    reg_lambda=1,
    objective='binary:logistic',
    scale_pos_weight=1)

In [95]:
make_submission(xgb_model)